In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from src.data import preprocessing
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unidecode

In [ ]:
params = preprocessing.dict_params
INPUT_FILE_PATH = './../data/raw'
OUTPUT_FILE_PATH = './../data/interim'
INDEX_KEY = 0 # the 1st column is the primary key

preprocessing.preprocess_dataset(preprocessing.get_param('synthse', 'file_name'), INPUT_FILE_PATH, OUTPUT_FILE_PATH, INDEX_KEY)

In [102]:
ingred_red = ingred_df.iloc[:, keep_ingred]

df1_key = ingred_red.columns[[0]].tolist()
pivot_column = ingred_red.columns[[1]].tolist()
value_column = ingred_red.columns[2]
print(df1_key, pivot_column, value_column)

pivot_df = pd.pivot(data=ingred_red, index=df1_key, columns=pivot_column, values = 'Score unique EF (mPt/kg de produit)').reset_index().rename_axis(None, axis=1)
pivot_df.fillna()




['Ciqual AGB'] ['Ingredients'] Score unique EF (mPt/kg de produit)


,Ciqual AGB,Abats de bœuf,Abats de porc,Abats de poulet,Abats de veau,Abricot,Ail,Amande,Amidon de maïs,Ananas,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
0,1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.079434,NaN,NaN,NaN,NaN,NaN
1,1012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.010289,0.073136,NaN,0.006898,NaN,NaN
4,1018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.089332,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,42604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1034,42605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1035,42606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1036,51550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def merge(df_left, df_righ, key_left, key_right, keep_left, keep_right):
    pd.merge(df_left)

In [34]:
synthese_df.shape

(2479, 27)

In [50]:
ingred_df.groupby('Ingredients').count()

,Ciqual AGB,Ciqual code,Nom Français,Groupe d'aliment,Sous-groupe d'aliment,LCI Name,Score unique EF (mPt/kg de produit),Changement climatique (kg CO2 eq/kg de produit),Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit),Rayonnements ionisants (kBq U-235 eq/kg de produit),...,Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
Ingredients,,,,,,,,,,,,,,,,,,,,,
Abats de bœuf,25,25,25,25,25,25,25,25,25,25,...,25,25,25,25,25,25,25,25,25,25
Abats de porc,50,50,50,50,50,50,50,50,50,50,...,50,50,50,50,50,50,50,50,50,50
Abats de poulet,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
Abats de veau,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
Abricot,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vin rouge,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
Yaourt,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
citron,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [38]:
ingred_df

,Ciqual AGB,Ciqual code,Nom Français,Groupe d'aliment,Sous-groupe d'aliment,LCI Name,Ingredients,Score unique EF (mPt/kg de produit),Changement climatique (kg CO2 eq/kg de produit),Appauvrissement de la couche d'ozone (E-06 kg CVC11 eq/kg de produit),...,Particules (E-06 disease inc./kg de produit),Acidification terrestre et eaux douces (mol H+ eq/kg de produit),Eutrophisation terreste (mol N eq/kg de produit),Eutrophisation eaux douces (E-03 kg P eq/kg de produit),Eutrophisation marine (E-03 kg N eq/kg de produit),Utilisation du sol (Pt/kg de produit),Écotoxicité pour écosystèmes aquatiques d'eau douce (CTUe/kg de produit),Épuisement des ressources eau (m3 depriv./kg de produit),Épuisement des ressources énergétiques (MJ/kg de produit),Épuisement des ressources minéraux (E-06 kg Sb eq/kg de produit)
0,11168,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Jaune d'œuf,0.039743,0.164754,0.014089,...,0.046616,0.006761,0.029441,0.052027,1.329230,16.984964,7.404112,0.337769,1.475548,0.251356
1,11168,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Ail,0.004417,0.011444,0.002512,...,0.000781,0.000105,0.000328,0.004760,0.189983,1.643787,1.216336,0.291209,0.229937,0.084028
2,11168,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Jus de citron,0.019375,0.061230,0.007842,...,0.006365,0.000951,0.003573,0.016918,0.213721,3.707242,2.625043,1.445978,0.793865,0.203959
3,11168,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Huile d'olive,0.445364,1.161825,0.136467,...,0.142041,0.021524,0.088459,0.554361,9.263219,7.579606,464.100950,9.374618,12.100965,4.424012
4,11168,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Autres étapes,0.080761,0.571172,0.126768,...,0.024611,0.002630,0.008368,0.090374,1.295442,4.529954,11.433712,0.759522,22.626111,1.278583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5666,19552,19552,"Yaourt à la grecque, sur lit de fruits",lait et produits laitiers,produits laitiers frais et assimilés,"Yogurt, Greek-style, on a bed of fruits",Fraise,0.121232,0.113532,0.637141,...,0.027982,0.000952,0.003176,0.035963,0.370286,3.561618,195.859930,1.702002,1.587775,3.330941
5667,19552,19552,"Yaourt à la grecque, sur lit de fruits",lait et produits laitiers,produits laitiers frais et assimilés,"Yogurt, Greek-style, on a bed of fruits",Autres étapes,0.070305,0.549462,0.115755,...,0.023034,0.002426,0.007428,0.080773,1.159934,4.096425,6.238759,0.248905,21.610336,1.176044
5668,19860,19860,"Yaourt à la grecque, nature",lait et produits laitiers,produits laitiers frais et assimilés,"Yogurt, Greek-style, plain",Lait,0.063101,0.693950,0.010273,...,0.043025,0.006846,0.029968,0.047026,8.579385,62.085522,1.178242,0.013914,1.062461,0.019234
5669,19860,19860,"Yaourt à la grecque, nature",lait et produits laitiers,produits laitiers frais et assimilés,"Yogurt, Greek-style, plain",Lait demi-écremé,0.106409,1.198262,0.031746,...,0.089744,0.013406,0.058485,0.107689,3.903060,66.972761,17.562407,0.373698,3.073724,0.642726
